In [16]:
from dotenv import load_dotenv
import os
import time
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np 
from Preprocessing.preprocessing_pipeline_final import preprocessing_pipeline

load_dotenv()  # lädt .env automatisch
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")



In [33]:
pd.set_option("display.max_columns", None)

In [ ]:
def call_llm(text):
    # Diese Funktion nutzt die API von Google um sich gegen das LLM zu schalten. Prompt etc. ist unten zu sehen

    llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                '''Du bist ein Experte für Automobilklassifikation. Deine Aufgabe ist es, Automodelle basierend auf ihrem Namen einem von fünf vordefinierten Fahrzeugsegmenten zuzuordnen. Bitte gib nur den Namen des passenden Segments zurück.

                    Die Segmente sind:

                    1. Kleinwagen – kleine Stadtautos, z.B. VW Up!, Renault Clio, Fiat Panda
                    2. Mittelklasse – normale Alltagsautos, z.B. VW Golf, Audi A4, BMW 3er
                    3. Geländewagen – große Fahrzeuge mit viel Platz, z.B. BMW X3, VW Tiguan, Ford Kuga, VW Multivan
                    4. Sportwagen – sportliche Fahrzeuge mit viel PS, z.B. Porsche 911, Audi R8, BMW M4
                    5. Luxusklasse – hochwertige Fahrzeuge mit Premiumausstattung, z.B. BMW 7er, Mercedes S-Klasse, Tesla Model S

                    Beispiele:
                    
                    Input: smart forTwo
                    Output: Kleinwagen

                    Input: Volkswagen Golf  
                    Output: Mittelklasse

                    Input: BMW X5  
                    Output: Geländewagen

                    Input: Porsche 911  
                    Output: Sportwagen

                    Input: Bentley Mulsanne
                    Output: Luxusklasse

                Gib mir als Output nur das Fahrzeugsegment aus.                             
                ''',
            ),
            ("human", "{input}"),
        ]
    )

    chain = prompt | llm
    res = chain.invoke(
        {
            "input": text
        }
    )

    return res.content


def use_llm_on_model(model_list):
    # Diese Funktion ruft die call_llm Funktion auf, um der Spalte model das Fahrzeugsegment zuzuordnen

    result = {}
    counter = 0
    limit_per_minute = 15 # brauchen diesen Timer um nicht zu viele Anfragen zu stellen (free sind 15 Anfragen/Min)

    for idx, model in enumerate(model_list, start=1):
        result[model] = call_llm(model)
        counter += 1

        # Warten, wenn das Limit erreicht ist
        if counter % limit_per_minute == 0:
            print(f"{counter} Anfragen gestellt – warte 60 Sekunden, um Rate Limit einzuhalten...")
            time.sleep(60)

    print("Fertig!")
    return result

In [39]:
X_train, X_test, y_train, y_test , X,y, categorical_features , numeric_features = preprocessing_pipeline()

In [ ]:
df = pd.concat([X, y], axis=1)

In [38]:
df.head()

,brand,model,color,year,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description,registration_month,registration_year,outlier_model_price,outlier_model_mileage,offer_description_cleaned,has_acc,has_additional_motortechnology,has_all_wheel_drive,has_assistence_systems,has_dab_radio,has_diesel_particel_feature,has_doubleclutch_transmission,has_heated_seats,has_leather_interior,has_navigation,has_panoramic_roof,has_sport_features,has_trailer_hitch,has_tuev,is_cabrio,is_combi,is_coupe,price_in_euro
322,alfa-romeo,Alfa Romeo Giulietta,grey,2010,125.0,170.0,Manual,Petrol,6.4,134.0,163870.0,Turismo 1.4 TB 16V MultiAir Navi-Leder,9,2010,False,False,turismo tb v multiair navileder,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,8970
323,alfa-romeo,Alfa Romeo Giulietta,grey,2010,125.0,170.0,Manual,Petrol,6.4,134.0,117000.0,Turismo (191),12,2010,False,False,turismo,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,6500
325,alfa-romeo,Alfa Romeo Giulietta,red,2010,125.0,170.0,Manual,Diesel,4.9,130.0,164940.0,Turismo/So+Wi.R/KLIMAAUT,11,2010,False,False,turismosowirklimaaut,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,7990
326,alfa-romeo,Alfa Romeo Giulietta,red,2010,88.0,120.0,Manual,Petrol,6.6,156.0,147866.0,Turismo (191),11,2010,False,False,turismo,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,6900
330,alfa-romeo,Alfa Romeo Giulietta,black,2010,88.0,120.0,Manual,Petrol,6.6,156.0,143500.0,Giulietta Turismo 1.4Bi- Xenon Tempomat 1.Hand,8,2010,False,False,giulietta turismo bi xenon tempomat hand,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,6650


In [ ]:
model_list = df['model'].unique().tolist()
result = use_llm_on_model(model_list)
df["segment"] = df["model"].map(result).fillna("unknown")

In [ ]:
df.to_csv('df_mit_segment.csv', index=False)